In [1]:
try:
    import nqcpfem
except ModuleNotFoundError as err:
    import sys
        # put path to nqcpfem module here   
    src_path = '/mnt/c/Users/olisc/NQCP/NQCP_FEM/'
    sys.path.append(src_path)
    import nqcpfem
import sympy
sympy.init_printing(use_latex='mathjax')
import IPython.display as disp
from matplotlib import pyplot as plt
import numpy as np
import os

import logging
logging.getLogger().setLevel(logging.INFO)



In [7]:
import numpy as np
import sympy
E0 = 1/(nqcpfem.UNIT_CONVENTION['J to eV']*1e6) # unit_scale: 10muev
from nqcpfem.solvers import PETScSolver
from nqcpfem.systems import PositionalState,DefiniteTensorComponent
from nqcpfem.envelope_function import RectangleDomain
from nqcpfem.fenics import FEniCsModel
from nqcpfem.band_model import LuttingerKohnHamiltonian
from nqcpfem.systems.dot_sc_dot import *
def system_builder():
    H = LuttingerKohnHamiltonian(spatial_dim=3).material_spec('Ge').add_z_confinement(1,'box',25e-9)
    H.BdG_extension()
    H.add_zeeman_term(B=[0,0,1e-1])
    

    #H.add_potential(-sympy.symbols('\mu'))

    domain =RectangleDomain(200e-9,100e-9,0)
    domain.resolution = [100,50]

    model = FEniCsModel(H,None, boundary_condition=0,function_class=('CG',1))
    ldot = Dot(-(75e-9+100e-9/2),4e11,0,150e-9,4e11,0,150e-9,)
    barr = Barrier(0e-9,100*E0)
    #rbarr = Barrier(15e-9,3/(nqcpfem.UNIT_CONVENTION['J to eV']*1000))
    sc = Superconductor(1000*E0,100e-9,50e-9,0,5*E0)
    syst = DotSCDot(model,ldot,barr,sc,domain_resolution=[100,100])

    mu,mu_sc = sympy.symbols('\mu,\mu_{sc}')
    chemical_potential = SymbolicFunction(sympy.Piecewise((-mu_sc,syst.domains['sc_in']),(-mu,True)),'\mu(x)')
    H.add_potential(chemical_potential)

    H.parameter_dict[mu] = 4020*E0 # will be set to something other than zero on determining couping
    H.parameter_dict[mu_sc] = 13e3*E0

    mu_L,mu_R = sympy.symbols('\mu_{L},\mu_{R}')
    detuning = SymbolicFunction(sympy.Piecewise((-mu_L,syst.domains['ld_in']),(-mu_R,syst.domains['rd_in']),(0,True)),'\mu_{lr}(x)')
    H.add_potential(detuning)
    H.parameter_dict[mu_L] = 0
    H.parameter_dict[mu_R] = 0
    return syst

syst = system_builder()
H = syst.envelope_model.band_model

In [14]:
from nqcpfem.functions import SymbolicFunction
V = list(H.function_dict.values())[-1].expression
syms = list(V.free_symbols)
syms,V

⎛                          ⎧-\mu_{L}  for x ≥ -2.0e-7 ∧ y ≥ -7.5e-8 ∧ x ≤ -5.0
⎜                          ⎪                                                  
⎜[x, y, \mu_{R}, \mu_{L}], ⎨-\mu_{R}   for x ≥ 5.0e-8 ∧ y ≥ -7.5e-8 ∧ x ≤ 2.0e
⎜                          ⎪                                                  
⎝                          ⎩   0                             otherwise        

e-8 ∧ y ≤ 7.5e-8⎞
                ⎟
-7 ∧ y ≤ 7.5e-8 ⎟
                ⎟
                ⎠

In [19]:
Vl = SymbolicFunction(V.subs({syms[-2]:0}),'V_{L}(x)')
Vr = SymbolicFunction(V.subs({syms[-1]:0}),'V_{R}(x)')

def potental_adder(v):
    arr = sympy.Array(np.diag([1,1,1,1,-1,-1,-1,-1]))
    return arr*v


Vl.expression,Vr.expression

⎛                                                                     ⎧   0   
⎜⎧-\mu_{L}  for x ≥ -2.0e-7 ∧ y ≥ -7.5e-8 ∧ x ≤ -5.0e-8 ∧ y ≤ 7.5e-8  ⎪       
⎜⎨                                                                  , ⎨-\mu_{R
⎜⎩   0                             otherwise                          ⎪       
⎝                                                                     ⎩   0   

   for x ≥ -2.0e-7 ∧ y ≥ -7.5e-8 ∧ x ≤ -5.0e-8 ∧ y ≤ 7.5e-8⎞
                                                           ⎟
}   for x ≥ 5.0e-8 ∧ y ≥ -7.5e-8 ∧ x ≤ 2.0e-7 ∧ y ≤ 7.5e-8 ⎟
                                                           ⎟
                          otherwise                        ⎠

In [ ]:
from nqcpfem.fenics import FEniCsObservable
FEM_Vr = FEniCsObservable(sympy.Array(np.eye(8)))

array([[ 1,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  1,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  1,  0,  0,  0,  0],
       [ 0,  0,  0,  0, -1,  0,  0,  0],
       [ 0,  0,  0,  0,  0, -1,  0,  0],
       [ 0,  0,  0,  0,  0,  0, -1,  0],
       [ 0,  0,  0,  0,  0,  0,  0, -1]])